In [1]:
import os

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

from models.xswin import XNetSwinTransformer, SwinResidualCrossAttention, _extract_windows


In [2]:
*A, B= 1, 2
A


[1]

In [3]:
x = torch.tensor([
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
    [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
])

x = torch.tensor([
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5],
])

# print(torch.cat((x, x), dim=-1))


# x = torch.vstack((torch.hstack((x, x*2)), torch.hstack((x*3, x*4))))
x = torch.stack((x, x*2, x*3, x*4))
x = torch.vstack((torch.hstack((x, x*2)), torch.hstack((x*3, x*4))))

# print(x)

# x = x.reshape(2, 2, -1)
x = x.unsqueeze(0)

print(x.shape)

x = _extract_windows(x, 4, 4)

# print(x.shape)

# print(x)


torch.Size([1, 8, 8, 5])


In [4]:
r1 = torch.randn((5, 10, 20, 64))
r2 = torch.randn((5, 10, 20, 64))

SRCA = SwinResidualCrossAttention(
    [4, 4],
    64,
    4,
)

SRCA(r1, r2).shape


AssertionError: torch.Size([5, 3, 5, 16, 64]) != torch.Size([5, 5, 9, 16, 64])

In [ ]:
patch_size = [4, 4]
embed_dim = 64
depths = [2, 2, 2]
num_heads = [4, 8, 16]
window_size = [4, 4]
num_classes = 1


swin = XNetSwinTransformer(patch_size, embed_dim, depths, 
                           num_heads, window_size, num_classes=num_classes,
                           final_downsample=False, residual_cross_attention=True,
                           )

IMG = 160
x = torch.randn((5, 3, IMG, IMG*2))
print(x.shape)

# print(swin)

y = swin(x)
print(y.shape)

print(swin)


torch.Size([5, 3, 160, 320])
torch.Size([5, 160, 320])
XNetSwinTransformer(
  (smooth_conv_in): ConvolutionTripletLayer(
    (layers): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.01)
      (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): LeakyReLU(negative_slope=0.01)
    )
  )
  (patching): Sequential(
    (0): Conv2d(64, 64, kernel_size=(4, 4), stride=(4, 4))
    (1): Permute()
    (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (encoder): ModuleList(
    (0): Sequential